In [ ]:
import numpy as np
from ase.build import bulk
from ase.io import read,write
from ase.io.trajectory import Trajectory
import py3Dmol
from qepy.calculator import QEpyCalculator
from qepy.io import QEInput

In [ ]:
from ase.units import kJ
from ase.eos import EquationOfState

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'",
        'outdir': "tmp",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'",
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al.pbe-nl-kjpaw_psl.1.0.0.UPF'],
    'k_points automatic': ['10 10 10 1 1 1'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [ ]:
atoms = bulk('Al', 'bcc', a=4.05)
write('Al.xyz',atoms)
v = py3Dmol.view(data=open('./Al.xyz','r').read(), style='sphere')
v.addUnitCell()
v.zoomTo()
v.show()

In [ ]:
qe_options = QEInput().update_atoms(atoms, qe_options = qe_options,  extrapolation=False)
QEInput().write_qe_input("file.in", qe_options=qe_options)

In [ ]:
!cat "file.in"

In [ ]:
atoms.calc = QEpyCalculator(qe_options=qe_options, logfile=True, extrapolation=False)

In [ ]:
%%time
traj = Trajectory('tmp.traj', 'w')
cell = atoms.get_cell()
volumes = []
energies = []

for x in np.linspace(0.72, 0.90, 8):
    atoms.set_cell(cell * x, scale_atoms=True)
    vol = atoms.get_volume()
    ene = atoms.get_potential_energy()
    volumes.append(vol)
    energies.append(ene)
    traj.write(atoms)

In [ ]:
from ase.units import kJ
from ase.eos import EquationOfState
eos = EquationOfState(np.array(volumes), np.array(energies))
v0, e0, B = eos.fit()
eos.plot('tmp.eos.png').title.get_text()